# Segmenting and Clustering Neighborhoods in the city of Toronto
### First part of the assignment
#### (Creating notebook, scraping data and transforming the dataframe)

Below the necesarry packages will be imported

In [1]:
import pandas as pd

***Scraping of the data***  
Pandas is used to scrape data form the wikipedia website listing the postal codes of Canada. This returns a dataframe consisting of 3 elements. The first element is the table we need.

In [30]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
df_scraped = pd.read_html(url)
df_toronto = df_scraped[0]
df_toronto.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


***Transforming the data***

In [32]:
#Drop unassigned Boroughs
unassigned_borrows = df_toronto[df_toronto["Borough"]== "Not assigned"].index
df_toronto.drop(unassigned_borrows,inplace = True)
df_toronto.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


_Note that the wikipedia table already has transformed their table in a way that postal codes stretching neighbourhoods display the corresponding neighbourhoods separated with a comma._  

Neighbourhoods that are not assigned will receive the name of their borough

In [33]:
unassigned_neighbourhoods = df_toronto[df_toronto["Neighbourhood"]== "Not assigned"].index
df_toronto.Neighbourhood[unassigned_neighbourhoods] = df_toronto.Borough[unassigned_neighbourhoods]
df_toronto.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


_Please note that on the table in wikipedia all the neighbourhoods have been assigned in the case that they have a borough assigned._